In [36]:
# Extract files in a subfolder
!echo "__TAR__B64__GOES__HERE__" \
    | base64 -d \
    | tar -xvzf -

select/Makefile
select/benchmark.c
select/select.c
select/test.c
select/test.h


In [3]:
!git clone https://github.com/google/benchmark.git

Cloning into 'benchmark'...
remote: Enumerating objects: 8850, done.
remote: Counting objects: 100% (1447/1447), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 8850 (delta 1293), reused 1274 (delta 1195), pack-reused 7403 (from 1)
Receiving objects: 100% (8850/8850), 2.80 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (5960/5960), done.


In [38]:
!cd benchmark && \
  git checkout 761305ec3b33abf30e08d50eb829e19a802581cc && \
  cmake -E make_directory "build" && \
  cmake -E chdir "build" cmake -DBENCHMARK_DOWNLOAD_DEPENDENCIES=on -DCMAKE_BUILD_TYPE=Release ../ && \
  cmake --build "build" --config Release

HEAD is now at 761305e Update user_guide.md (#1863)
-- Failed to find LLVM FileCheck
-- Google Benchmark version: v1.9.0-12-g761305ec, normalized to 1.9.0.12
-- Performing Test HAVE_STD_REGEX -- success
-- Performing Test HAVE_GNU_POSIX_REGEX -- failed to compile
-- Performing Test HAVE_POSIX_REGEX -- success
-- Performing Test HAVE_STEADY_CLOCK -- success
-- Performing Test HAVE_PTHREAD_AFFINITY -- success
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Looking for Google Test sources
-- Looking for Google Test sources in /content/benchmark/googletest
CMake Warning at CMakeLists.txt:37 (message):
  Did not find Google Test sources! Fetching from web...


-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have b

In [29]:
!cd select && make test

g++ -O0 -fno-omit-frame-pointer -fsanitize=undefined -fsanitize=address -g select.c test.c -o test.out
./test.out 
PASS '(less than 5)'
    expected 50, got deterministic 50 and rand 50, sorted arr: {3, 12, 50, 84, }
PASS '(len % 5 == 1)'
    expected 50, got deterministic 50 and rand 50, sorted arr: {2, 3, 9, 12, 50, 84, }
PASS '(len % 5 == 2)'
    expected 17, got deterministic 17 and rand 17, sorted arr: {3, 7, 11, 12, 15, 17, 23, 50, 60, 81, 84, 98, }
PASS '(len % 5 == 3)'
    expected 23, got deterministic 23 and rand 23, sorted arr: {2, 3, 7, 9, 11, 12, 15, 23, 50, 60, 81, 84, 98, }
PASS '(len % 5 == 4)'
    expected 9, got deterministic 9 and rand 9, sorted arr: {2, 3, 7, 9, 11, 12, 15, 17, 23, 50, 60, 81, 84, 98, }
PASS 'ends up in the lower part of recursion'
    expected 7, got deterministic 7 and rand 7, sorted arr: {2, 3, 6, 7, 9, 11, 12, 15, 17, 23, 40, 50, 60, 81, 84, 98, }
PASS 'ends up in the higher part of recursion'
    expected 60, got deterministic 60 and rand 60, s

In [ ]:
import json
import matplotlib.pyplot as plt

!cd select && make benchmark
print("")

# Load the JSON data
with open('select/benchmark.json', 'r') as f:
    data = json.load(f)

# Filter benchmark where run_type is iteration
benchmarks = [benchmark for benchmark in data['benchmarks'] if benchmark['run_type'] == 'iteration']

# Extract the relevant data for plotting
plots = {}
for benchmark in benchmarks:
    name = benchmark['name'].split('/')[0]
    N = int(benchmark['name'].split('/')[-1])

    if not name in plots:
      plots[name] = {}
      plots[name]['name'] = name
      plots[name]['x_values'] = [N]
      plots[name]['y_values'] = [float(benchmark['cpu_time'])]
    elif(N not in plots[name]['x_values']):
        plots[name]['x_values'].append(N)
        plots[name]['y_values'].append(float(benchmark['cpu_time']))

plt.figure(figsize=(10, 6))
for name, plot in plots.items():
    plt.plot(plot['x_values'], plot['y_values'], label=name)
plt.xlabel('Benchmark N')
plt.ylabel('CPU Time (ns)')
plt.title('Benchmark Results')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()